<a href="https://colab.research.google.com/github/xprilion/gemini-as-a-judge-for-rag-evals/blob/main/Step_1_Problem_Context_The_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gemini As A Judge for RAG Evals

## The RAG

### 1. Load the datasets

In [1]:
!wget https://raw.githubusercontent.com/567-labs/systematically-improving-rag/refs/heads/main/cohort_1/week1_bootstrap_evals/reviews.json

--2025-03-01 00:26:04--  https://raw.githubusercontent.com/567-labs/systematically-improving-rag/refs/heads/main/cohort_1/week1_bootstrap_evals/reviews.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 507865 (496K) [text/plain]
Saving to: ‘reviews.json’

reviews.json        100%[===================>] 495.96K  2.73MB/s    in 0.2s    

2025-03-01 00:26:05 (2.73 MB/s) - ‘reviews.json’ saved [507865/507865]



In [9]:
# !wget https://raw.githubusercontent.com/567-labs/systematically-improving-rag/refs/heads/main/cohort_1/week1_bootstrap_evals/synthetic_eval_dataset.json

### Packages

In [51]:
%%capture
!pip install qdrant-client[fastembed]

### Imports

In [53]:
import pandas as pd
import json
import os
import time
from tqdm import tqdm
from google import genai
from google.genai import types
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
import uuid

from google.colab import userdata

### Helpers

In [66]:
collection_name = "product_reviews"

In [67]:
GEMINI_KEY = userdata.get('GEMINI_API_KEY')
gemini_client = genai.Client(
    api_key=GEMINI_KEY
)

In [68]:
def getGeminiResponse(prompt, max_tokens=8192, response_type="text/plain"):
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(
                    text=prompt
                ),
            ],
        ),
    ]
    generate_content_config = types.GenerateContentConfig(
        temperature=0,
        top_p=0.95,
        top_k=40,
        max_output_tokens=max_tokens,
        response_mime_type=response_type,
    )
    response = gemini_client.models.generate_content(
        model="gemini-2.0-flash", contents=contents, config=generate_content_config
    )
    return response.text

In [69]:
getGeminiResponse("What is 2+3?")

'2 + 3 = 5\n'

### EDA

In [6]:
df = pd.read_json('reviews.json')

In [7]:
df.head()

,product_title,product_description,review
0,Hammer,This 16 oz claw hammer is perfect for general ...,I've been using this hammer for a few months n...
1,Hammer,This 16 oz claw hammer is perfect for general ...,This hammer is a solid addition to my toolbox....
2,Hammer,This 16 oz claw hammer is perfect for general ...,I purchased this hammer for some home renovati...
3,Hammer,This 16 oz claw hammer is perfect for general ...,"As a professional carpenter, I rely on my tool..."
4,Hammer,This 16 oz claw hammer is perfect for general ...,This hammer is a great value for the price. Th...


### Connect Qdrant

In [70]:
QDRANT_URL = "https://qdrant-1.sg-1.cloudtop.dev"
QDRANT_KEY = userdata.get('PERSONAL_QDRANT_KEY')

In [71]:
qdrant_client = QdrantClient(url=QDRANT_URL, api_key=QDRANT_KEY, port=None)

### Create Documents

In [32]:
documents = []
metadatas = []
ids = []

In [33]:
for index, row in df.iterrows():
    product_title = row['product_title']
    product_description = row['product_description']
    review = row['review']

    # Combine product information and review into a single document
    document = f"Title: {product_title}\nDescription: {product_description}\nReview: {review}"

    # Create metadata dictionary
    metadata = {
        "product_title": product_title,
        "product_description": product_description,
        "review": review,
        "index": index, # add index for reference
    }

    # Generate a unique ID
    doc_id = str(uuid.uuid4())

    documents.append(document)
    metadatas.append(metadata)
    ids.append(doc_id)

In [34]:
len(documents)

900

### Load data into qdrant

In [39]:
qdrant_client.add(
    collection_name=collection_name,
    documents=documents,
    metadata=metadatas,
    ids=ids
)

['d869d1c9-a78e-4765-93e2-dc008b4691bc',
 'cc0e3859-a9bd-4aa9-b158-d3c71efba145',
 'a45e79a3-afa6-4cb2-9899-1c8fcbf2ac35',
 '58fea7bf-5f95-44c8-b007-6420345ca0de',
 '70456f6b-537a-4ef1-bb75-3df239cad5f4',
 'de7d2def-9869-4b0f-a476-2ff68ade72c3',
 'f3fc4fe6-a566-4b6d-9efd-2b1f894dfa9c',
 '4150bcac-1448-4229-bac2-5d8d2f4dc380',
 'e1d0f510-57f6-49a1-b829-282e2c5b8b5f',
 '88eb3ec3-d24f-4dd4-b9e9-337826df7fab',
 '7f122fe2-4d62-4863-9612-e2cc17386780',
 '51b529b9-0369-44dc-978a-b9803298508e',
 '24571960-ab2a-4017-a2e0-8bd7676d34fd',
 '0bed9d16-19b0-4b4c-b877-3d59c71954a8',
 '1785af3e-1700-4097-a5f2-e923211a1987',
 'a96b656d-6994-4993-8c36-ff670a4a07a0',
 '3882a3ef-dd29-4ca9-98f7-2604f2c0e782',
 'b2874ad3-82af-48e5-8d37-20fe041bbc58',
 '19c90191-3a9f-4936-a7c2-4b837625f1da',
 '9f72f395-7de0-466a-918c-811a4e9f2f94',
 'f8c381b0-082b-4717-980d-cf1204a33229',
 '5718f503-6238-4c61-9ac0-b6a59f2a5128',
 '8d1acb82-48a9-4c67-8f4c-80271ec2d2fb',
 '88ceda0d-7ffe-43d7-a00b-02980f3853b3',
 '66e57cc9-2311-

### Test Retrieval

In [78]:
user_query = "I want to hang some shelves with a hammer"

In [79]:
search_result = qdrant_client.query(collection_name=collection_name, query_text=user_query)

In [80]:
for result in search_result:
    print(result.document)
    print("---")

Title: Hammer
Description: A lightweight 12 oz hammer ideal for small household repairs. Its compact size makes it easy to store and handle.
Review: I've been using this hammer for a few months now, and I'm very impressed with its performance. The 12 oz weight is ideal for small tasks, and the compact size makes it easy to store in my tool bag. The handle is ergonomic and provides a secure grip, even when I'm working in tight spaces. The hammer is also very durable; it has withstood several drops and still works perfectly. Great value for the price.
---
Title: Hammer
Description: A lightweight 12 oz hammer ideal for small household repairs. Its compact size makes it easy to store and handle.
Review: I bought this hammer for my daughter, who recently moved into her first apartment. She loves it! The lightweight design makes it easy for her to handle, and the compact size means she can store it in her small tool kit. The hammer is sturdy and well-made, and the grip is comfortable. She's 

In [81]:
def getRagResponse(question):
    search_result = qdrant_client.query(collection_name=collection_name, query_text=question)
    system_prompt = """
      You are an intelligent assistant designed to provide accurate and informative answers based on retrieved documents.

      Your primary task is to:

      Understand the user's query.
      Retrieve relevant information from the provided context (documents).
      Synthesize the retrieved information into a coherent and accurate response.

      documents:

      """

    documents_text = ""

    doc_count = 1
    for result in search_result:
      documents_text += str(doc_count) + ": \n" + result.document + "\n\n"
      doc_count += 1

    users_query = "\n\n The user is asking: " + question

    prompt = system_prompt + documents_text + users_query

    response = getGeminiResponse(prompt)

    return response

### Test RAG

In [82]:
getRagResponse(user_query)

'Based on the provided documents, the 12 oz hammer is highly recommended for hanging shelves. The reviews consistently praise its lightweight design, comfortable grip, and compact size, making it ideal for small household tasks like hanging shelves. Users also mention using it for assembling furniture, hanging pictures, and other DIY projects.\n'